- Step 1: Import Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings("ignore")


In [7]:
# Load the labeled dataset
df = pd.read_csv("CSV_Files/Training and Testing Sets/UNSW_NB15_training-set.csv", low_memory=False)
df.head()
print(df['label'].value_counts(normalize=True))


label
1    0.680622
0    0.319378
Name: proportion, dtype: float64


In [8]:
# Define the 8 user-friendly features
features = ['dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'smean', 'dmean', 'service']
target = 'label'

# Drop any rows with missing required columns
df = df.dropna(subset=features + [target])

# Encode 'service' column (must match app.py encoding)
service_mapping = {'-': 0, 'dns': 1, 'ftp': 2, 'http': 3, 'smtp': 4, 'ssh': 5, 'ssl': 6}
df['service'] = df['service'].map(service_mapping).fillna(0).astype(int)



# Prepare X and y
X = df[features]
y = df[target]

# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train model with balanced class weights (recommended)
model = RandomForestClassifier(
    n_estimators=150,
    max_depth=15,
    min_samples_leaf=5,
    class_weight='balanced_subsample',
    random_state=42
)

model.fit(X_train, y_train)

# Optional: Evaluate performance
print(classification_report(y_test, model.predict(X_test)))


              precision    recall  f1-score   support

           0       0.87      0.95      0.91     11169
           1       0.98      0.94      0.96     23900

    accuracy                           0.94     35069
   macro avg       0.92      0.94      0.93     35069
weighted avg       0.94      0.94      0.94     35069



In [9]:
import joblib

# Save model and scaler
joblib.dump(model, "IsAffected/rf_model.pkl")
joblib.dump(scaler, "IsAffected/scaler.pkl")


['IsAffected/scaler.pkl']